In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

# change path if necessary
import sys
my_path = r'C:\Users\utente\Documents\GitHub\Project2'
sys.path.insert(0,my_path + r'/code/Flavio')

import matplotlib.pyplot as plt
import numpy as np
import os
import tensorflow as tf
print('TensorFlow imported')

import dictionary_helpers as dh

# imports
import numpy as np 
import matplotlib.pyplot as plt

In [ ]:
import os
os.chdir(r'C:\Users\utente\Desktop\data\twitter_datasets_stanford')

glove_dict = dh.build_glove_dict('glove.twitter.27B.25d.txt')

print(len(glove_dict))


In [3]:
glove_dict[',']

array([ 0.84705 , -1.0349  , -0.050419,  0.27164 , -0.58659 ,  0.99514 ,
        0.25267 ,  1.6963  ,  0.10313 ,  0.80073 ,  0.74655 , -1.2667  ,
       -4.036   , -0.22557 ,  0.16322 , -0.67015 , -0.64812 ,  0.010373,
       -0.71889 , -0.74997 ,  0.24862 ,  0.10319 , -1.1732  ,  0.58196 ,
        0.33846 ])

In [4]:
import os
os.chdir(r'C:\Users\utente\Desktop\data\twitter_datasets_epfl\short')
#os.chdir(r'C:\Users\utente\Desktop\data\twitter_datasets_epfl\full')

tweets = dh.build_tweets_matrix('all_short_processed.txt')
        
print(len(tweets))

191321


In [5]:
tweets[1][1]

'your'

In [6]:
x = np.array([])

ciao=np.append(x,[1,4], axis=0)


print(ciao)

[ 1.  4.]


### Build averages+min+max

In [7]:
feat_size = 25

In [8]:
X = np.empty([len(tweets), feat_size*3])

for i in range(len(tweets)):
    feats = []
    for j in range(len(tweets[i])):
        if tweets[i][j] in glove_dict:
            feats.append(glove_dict[tweets[i][j]])
        else:
            feats.append([0]*feat_size)

    X[i]= np.hstack((np.mean(feats, axis=0), np.amax(feats, axis=0), np.amin(feats, axis=0)))

print(X.shape)
print(X[9])

labels = []

for i in range(90233):
    labels.append([1,0])
for i in range(len(tweets)-90233):
    labels.append([0,1])


(191321, 75)
[-0.1865898  0.236394   0.330381  -0.508172  -0.6313082 -0.1213108
  1.256084  -0.3280116 -0.69894    0.190782  -0.289562   0.6072632 -5.21422
 -0.1280236 -0.211636   0.387608   0.2012168 -0.0554106 -0.053988
 -0.2539778  0.139662   0.33784   -0.0941022 -0.0355024  0.0722898
  0.088601   0.59108    0.96279    0.19839   -0.092441   0.38567    2.1
  0.066642  -0.49794    0.70047    0.43174    1.4784    -3.1591     0.15437
  0.52152    0.86374    0.59554    0.88961    1.292      0.33094    0.4516
  0.65844    0.54594    0.25164    0.2758    -0.45649   -0.20207   -0.12672
 -1.1372    -1.247     -0.40767    0.68192   -0.84406   -1.1467    -0.17034
 -0.87221   -0.013954  -5.995     -0.43106   -0.9154    -0.11314   -0.33704
 -0.47587   -0.84178   -0.84126   -0.4582    -0.16727   -0.47606   -0.39695
 -0.16073  ]


In [9]:
featSet = []

for i in range(len(tweets)):
    featSet.append([X[i,:].tolist(), labels[i]])

In [10]:
print(featSet[190000])

[[0.16076666666666664, 0.37080678666666667, -0.15070153333333333, -0.031043000000000022, -0.4298712, -0.30033666666666664, 1.0273436666666667, 1.2202581333333336, -0.8047160000000002, 0.14191302, -0.20511066666666664, 0.07287840000000001, -3.885906666666666, -0.49186399999999997, -0.16666466666666666, -0.22948193333333336, 0.11266059999999999, -0.6132825333333332, -0.6732669333333333, -0.05011833333333333, 0.213364, -0.29275606666666665, -0.08284273333333335, 0.4607126666666667, 0.2680893333333333, 0.62415, 1.1861, 0.61622, 1.0119, 0.084332, 0.12081, 2.2315, 2.6356, 0.0, 0.70047, 0.57517, 1.0839, 0.0, 0.0, 0.31843, 1.3491, 0.57602, 0.0, 0.45495, 0.65635, 0.81819, 0.53068, 1.6392, 1.6294, 1.6189, -0.62645, -0.29194, -1.3632, -1.201, -0.97979, -1.459, -0.025355, -0.73405, -1.7277, -1.1867, -0.8847, -2.0128, -6.0385, -1.2343, -0.94359, -0.94879, -0.71249, -0.9871, -1.3348, -0.76565, -0.55258, -1.1655, -0.62747, -1.6099, -0.88679], [0, 1]]


In [11]:
del(X)
del(labels)
del(tweets)
del(glove_dict)

#### crea train_x train_y test_x test_y

In [26]:
import random

random.shuffle(featSet)

featSet = np.array(featSet)

testing_size = int(0.1*len(featSet))

train_x = list(featSet[:,0][:-testing_size])
train_y = list(featSet[:,1][:-testing_size])

test_x = list(featSet[:,0][-testing_size:-1])
test_y = list(featSet[:,1][-testing_size:-1])


In [29]:
n_inputs = 75
n_nodes_hl1 = 75
n_nodes_hl2 = 30
n_nodes_hl3 = 15

n_classes = 2
batch_size = 250


# input feature size = 50feats x 20words = 2000 
x = tf.placeholder('float', [None, n_inputs])
y = tf.placeholder('float')

#DEFINITION OF THE NETWORK:
#1) random initialization of the various layer with respectives dimensions, the layer is a DICTIONARY of weigths and biases
#2) application of weights and biases and activation function to each layer


def neural_network_model(data):
    # input_data * weights + biases

    #tf.random_normal returns an array of the specified dimension filled with randomly distributed numbers
    #tf.Variable = object to be created that works in the tensorflow session, just a wrapper.
    hidden_l1 = {'weights': tf.Variable(tf.random_normal([n_inputs, n_nodes_hl1])),
    'biases': tf.Variable(tf.random_normal([n_nodes_hl1]))}

    hidden_l2 = {'weights': tf.Variable(tf.random_normal([n_nodes_hl1, n_nodes_hl2])),
    'biases': tf.Variable(tf.random_normal([n_nodes_hl2]))}

    hidden_l3 = {'weights': tf.Variable(tf.random_normal([n_nodes_hl2, n_nodes_hl3])),
    'biases': tf.Variable(tf.random_normal([n_nodes_hl3]))}

    output_l  = {'weights': tf.Variable(tf.random_normal([n_nodes_hl3, n_classes])),
    'biases': tf.Variable(tf.random_normal([n_classes]))}

    l1 = tf.add(tf.matmul(data, hidden_l1['weights']), hidden_l1['biases'])
    l1 = tf.nn.relu(l1)

    l2 = tf.add(tf.matmul(l1, hidden_l2['weights']), hidden_l2['biases'])
    l2 = tf.nn.relu(l2)

    l3 = tf.add(tf.matmul(l2, hidden_l3['weights']), hidden_l3['biases'])
    l3 = tf.nn.relu(l3)

    output = tf.add(tf.matmul(l3, output_l['weights']), output_l['biases'])
    
    return output

def train_neural_network(x):
    prediction = neural_network_model(x)
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=prediction, labels=y))   # v1.0 changes
    # optimizer value = 0.001, Adam similar to SGD
    # everything is updating through this optimizer, i don't know how.... problem!? maybe better not to know
    optimizer = tf.train.AdamOptimizer().minimize(cost)
    epochs_no = 50
 
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())   # v1.0 changes
  
        # training
        for epoch in range(epochs_no):
            epoch_loss = 0

            #batching
            i = 0
            while i < len(train_x):
                start = i
                end = i+batch_size
                
                batch_x = np.array(train_x[start:end])
                batch_y = np.array(train_y[start:end])
                
                _, c = sess.run([optimizer, cost], feed_dict = {x: batch_x, y: batch_y})
                # code that optimizes the weights & biases
                epoch_loss += c
                i+=batch_size
                
            print('Epoch', epoch+1, 'completed out of', epochs_no, 'loss:', epoch_loss)

        # testing
        correct = tf.equal(tf.argmax(prediction, 1), tf.argmax(y, 1))
        accuracy = tf.reduce_mean(tf.cast(correct, 'float'))
        print('Accuracy:', accuracy.eval({x: test_x, y: test_y}))

train_neural_network(x)

Epoch 1 completed out of 50 loss: 3957.93492892
Epoch 2 completed out of 50 loss: 344.349525378
Epoch 3 completed out of 50 loss: 190.23245853
Epoch 4 completed out of 50 loss: 130.749658352
Epoch 5 completed out of 50 loss: 91.6765143982
Epoch 6 completed out of 50 loss: 67.9285935868
Epoch 7 completed out of 50 loss: 50.563290829
Epoch 8 completed out of 50 loss: 40.0175684578
Epoch 9 completed out of 50 loss: 32.2218884737
Epoch 10 completed out of 50 loss: 25.9500266801
Epoch 11 completed out of 50 loss: 22.5057114224
Epoch 12 completed out of 50 loss: 18.3631578137
Epoch 13 completed out of 50 loss: 16.8957597028
Epoch 14 completed out of 50 loss: 14.3127698676
Epoch 15 completed out of 50 loss: 11.9556188734
Epoch 16 completed out of 50 loss: 11.4944361826
Epoch 17 completed out of 50 loss: 9.82760720857
Epoch 18 completed out of 50 loss: 9.68349231366
Epoch 19 completed out of 50 loss: 8.71518126732
Epoch 20 completed out of 50 loss: 7.45344034489
Epoch 21 completed out of 50 lo

## TESTTTTT